In [1]:
import sys, os, gzip, pickle
sys.path.append('..')
import torch
import numpy as np
from torch.utils.data import DataLoader
from src.networks import DNNBlock, BaseNetwork
from src.data import feature_to_composition, BaseDataset, CompositionData
from src.trainer import DefaultTrainer

class AutoEncoder(BaseNetwork):
    def __init__(self,
                 input_dim:int, 
                 latent_dim:int, 
                 encoder_hidden_dim:int = 32,
                 encoder_hidden_layers:int = 2,
                 decoder_hidden_dim:int = 32,
                 decoder_hidden_layers:int = 2,
                 batch_norm:bool = True, 
                 dropout:float = 0,
                 activation:str = 'LeakyReLU',
                 **kwargs): 
        super(AutoEncoder, self).__init__()
        self._model_param = {
            'input_dim':input_dim,
            'latent_dim':latent_dim,
            'encoder_hidden_dim':encoder_hidden_dim,
            'encoder_hidden_layers':encoder_hidden_layers,
            'decoder_hidden_dim':decoder_hidden_dim,
            'decoder_hidden_layers':decoder_hidden_layers,
            'batch_norm':batch_norm,
            'dropout':dropout,
            'activation':activation,
        }
        self.comp_encoder = DNNBlock(input_dim, latent_dim, encoder_hidden_dim, 
                                     encoder_hidden_layers, batch_norm, dropout, activation)
        self.comp_decoder = DNNBlock(latent_dim, input_dim, decoder_hidden_dim, 
                                     decoder_hidden_layers, batch_norm, dropout, activation)

    def forward(self, x):
        l = self.comp_encoder(x)
        y = torch.nn.Sigmoid()(self.comp_decoder(l))
        return l, y

    def save(self, path, prefix, overwrite=True):
        self._save(path, f'{prefix}_full.model', overwrite)
        self.comp_encoder._save(path, f'{prefix}_encoder.model', overwrite)
        self.comp_decoder._save(path, f'{prefix}_decoder.model', overwrite)


def cfn(dataset):
    feat = []
    info = []
    for data in dataset:
        feat.append(data.comp_feat)
        info.append(data.to_dict())
    return torch.vstack(feat), info

In [2]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.sampler import SubsetRandomSampler
import time, pickle

DS = CompositionDataset()
DS.from_file()
DS.to('cuda')

target_test_mask = DS._year > 2018
target_valid_mask = DS._year == 2018
target_train_mask = DS._year < 2018

train_dl = DataLoader(DS, batch_size=128, 
                      sampler=SubsetRandomSampler(np.where(target_train_mask)[0]))
valid_dl = DataLoader(DS, batch_size=4096, 
                      sampler=np.where(target_valid_mask)[0])
test_dl = DataLoader(DS, batch_size=4096, 
                     sampler=np.where(target_test_mask)[0])

target_train_mask.sum(), target_valid_mask.sum(), target_test_mask.sum()

(24120, 1988, 2485)

In [3]:
torch.set_float32_matmul_precision('high')

- check performance

In [5]:
import time
model = AutoEncoder(104, 16, encoder_hidden_dim=64, encoder_hidden_layers=4, 
                    decoder_hidden_dim=64, decoder_hidden_layers=4)
#model_c = torch.compile(model)
#trainer = AETrainer(model_c, 1e-4, device='cuda')
trainer = AETrainer(model, 1e-4, device='cuda')

ts = []
for i in range(50):
    t0 = time.time()
    trainer.train(train_dl)
    t1 = time.time()
    trainer.test(train_dl)
    t2 = time.time()
    trainer.test(valid_dl)
    t3 = time.time()
    trainer.test(test_dl)
    t4 = time.time()
    ts.append([t1-t0, t2-t1, t3-t2, t4-t3])
np.mean(ts, 0), np.std(ts, 0)

with open('../dump/time_torch_v2c.pkl','wb') as f:
    pickle.dump(np.array(ts), f)

(array([0.47668373, 0.2887895 , 0.00311273, 0.00206484]),
 array([7.31276733e-02, 3.24810087e-02, 1.13259765e-03, 5.64105847e-05]))

In [13]:
_, out = trainer.test(test_dl)
(out['pred'] * out['target']).sum(1).mean(), np.square(out['pred']-out['target']).sum(1).mean()

(0.9595169, 3.4811459)

In [20]:
for latent_dim in [8, 16, 32, 64]:
    best_loss = 1e5
    model = AutoEncoder(104, latent_dim, encoder_hidden_dim=64, encoder_hidden_layers=4, 
                        decoder_hidden_dim=64, decoder_hidden_layers=4)
    trainer = AETrainer(model, 1e-4, device='cuda')
    output_path = f'/home/jhyang/WORKSPACES/MODELS/isyn/AE/train_unique_nn_latent_{latent_dim:02d}'
    os.makedirs(output_path, exist_ok=True)
    writer = SummaryWriter(output_path)
    for epoch in range(50000):
        train_loss = trainer.train(train_dl)
        valid_loss, valid_output = trainer.test(valid_dl)
        writer.add_scalar('Loss/Train', train_loss, epoch+1)
        writer.add_scalar('Loss/Valid', valid_loss, epoch+1)
        if (epoch + 1)%10 == 0:
            test_loss, test_output = trainer.test(test_dl)
            writer.add_scalar('Loss/Test', test_loss, epoch+1)
            
            if (epoch+1)%1000 == 0:
                trainer.model.save(output_path, f'{epoch+1:05d}.model')
                with open(os.path.join(output_path, f'{epoch+1:05d}.output.valid.pkl'),'wb') as f:
                    pickle.dump(valid_output, f)
                with open(os.path.join(output_path, f'{epoch+1:05d}.output.test.pkl'),'wb') as f:
                    pickle.dump(test_output, f)

        if valid_loss < best_loss:
            best_loss = valid_loss
            trainer.model.save(output_path, 'best.model')
            with open(os.path.join(output_path, 'best.output.valid.pkl'),'wb') as f:
                pickle.dump(valid_output, f)
            test_loss, test_output = trainer.test(test_dl)
            with open(os.path.join(output_path, 'best.output.test.pkl'),'wb') as f:
                pickle.dump(test_output, f)
            with open('best.epoch.txt','w') as f: f.write(str(epoch+1))

        print(f'{epoch:6d} / {train_loss:10.4e} / {best_loss:10.4e} / {valid_loss:10.4e} / {test_loss:10.4e}', end='\r')
    print()

In [14]:
with open(os.path.join(output_path, 'best.output.test.pkl'),'rb') as f:
    j = pickle.load(f)

In [8]:

model = AutoEncoder(104, 16, encoder_hidden_dim=64, decoder_hidden_dim=64, encoder_hidden_layers=3, decoder_hidden_layers=3)
trainer = AETrainer(model, 1e-4, device='cuda')

output_path = '../dump/test/ae/target_only'
os.makedirs(output_path, exist_ok=True)
writer = SummaryWriter(output_path)

t = []
for epoch in range(10000):
    t0 = time.time()
    train_loss = trainer.train(train_dl)
    t1 = time.time()
    valid_loss, _ = trainer.test(valid_dl)
    t2 = time.time()
    test_loss, _ = trainer.test(test_dl)
    t3 = time.time()
    t.append([t1-t0, t2-t1, t3-t2])
    writer.add_scalar('Loss/train', train_loss, epoch+1)
    writer.add_scalar('Loss/valid', valid_loss, epoch+1)
    writer.add_scalar('Loss/test', test_loss, epoch+1)
    print('{:7.3f} {:7.3f} {:7.3f}'.format(*np.mean(t, 0)), end='\r')
# np.array(t)    

KeyboardInterrupt: 

In [ ]:
torch.utils.b

In [ ]:
import os
output_path = '../dump/test/ae/with_precursor'
os.makedirs(output_path, exist_ok=True)
writer = SummaryWriter(output_path)
for epoch in range(2000):
    train_loss = trainer.train(train_all_dl)
    valid_loss, _ = trainer.test(valid_dl)
    test_loss, _ = trainer.test(test_dl)
    writer.add_scalar('Loss/train', train_loss, epoch+1)
    writer.add_scalar('Loss/valid', valid_loss, epoch+1)
    writer.add_scalar('Loss/test', test_loss, epoch+1)
    

In [41]:
new_trainer = AETrainer(trainer.model, 1e-5)
_, out = new_trainer.test(test_dl)

In [57]:
#out[1]['comp_feat'], {k:v for k,v in sorted(out[1]['comp_pred'].items(), key=lambda x:x[1], reverse=True)}
#out[0]['latent']
z = torch.rand((10,5))
k = z / z.sum(1).view(-1,1)
#z.sum(0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])